In [1]:
import pandas as pd
from FRA import *

In [2]:
data = pd.read_csv('../Data/performances.csv')
columnas = ['Is_there', 'Dyad', 'Player', 'Round', 'Score']
columnas1 = columnas + ['a'+str(i)+str(j) for i in range(1,9) for j in range(1,9)]
data = data[columnas1]
data['Player'] = data['Player'].apply(lambda x: x[-3:])
data.head(2)

,Is_there,Dyad,Player,Round,Score,a11,a12,a13,a14,a15,...,a77,a78,a81,a82,a83,a84,a85,a86,a87,a88
0,Unicorn_Absent,303-869,PL1,1,17,1,1,1,1,1,...,1,1,1,0,0,0,0,0,1,1
1,Unicorn_Absent,303-869,PL2,1,17,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0


In [3]:
def region2vec(x):
    columnas = ['a'+str(i)+str(j) for i in range(1,9) for j in range(1,9)]
    vector = []
    for c in columnas:
        vector.append(x[c])
    return vector

data['Absent1'] = data.groupby(['Dyad', 'Player'])['Is_there'].transform('shift',-1)
data['Region'] = data.apply(region2vec, axis=1)
data['RegionGo'] = data.groupby(['Dyad', 'Player'])['Region'].transform('shift',-1)
data = data[['Absent1'] + columnas + ['Region', 'RegionGo']]
data.head(2)

,Absent1,Is_there,Dyad,Player,Round,Score,Region,RegionGo
0,Unicorn_Present,Unicorn_Absent,303-869,PL1,1,17,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,Unicorn_Present,Unicorn_Absent,303-869,PL2,1,17,"[0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
data.shape

(5400, 8)

In [5]:
df = data[(data['Absent1']=='Unicorn_Absent') & (data['Is_there']=='Unicorn_Absent')]
df.shape

(1244, 8)

In [6]:
df.sort_values(by=['Dyad', 'Round', 'Player'], inplace=True)
df['Region2'] = df.groupby(['Dyad', 'Round'])['Region'].transform('shift',1)
df['Region1'] = df.groupby(['Dyad', 'Round'])['Region'].transform('shift',-1)
df['RegionOtro'] = df.apply(lambda x: x['Region1'] if x['Player']=='PL1' else x['Region2'], axis=1)
df['Overlap'] = df.apply(lambda x: np.multiply(x['Region'], x['RegionOtro']), axis=1)
del df['Region1']
del df['Region2']
del df['RegionOtro']
df.head()

<ipython-input-6-68f2e9a90ae7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=['Dyad', 'Round', 'Player'], inplace=True)
<ipython-input-6-68f2e9a90ae7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Region2'] = df.groupby(['Dyad', 'Round'])['Region'].transform('shift',1)
<ipython-input-6-68f2e9a90ae7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

,Absent1,Is_there,Dyad,Player,Round,Score,Region,RegionGo,Overlap
5040,Unicorn_Absent,Unicorn_Absent,140-615,PL1,1,23,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5041,Unicorn_Absent,Unicorn_Absent,140-615,PL2,1,23,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5056,Unicorn_Absent,Unicorn_Absent,140-615,PL1,9,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5057,Unicorn_Absent,Unicorn_Absent,140-615,PL2,9,27,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5059,Unicorn_Absent,Unicorn_Absent,140-615,PL1,10,32,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
focals, estrategias = create_regions_and_strategies(8)
df['Sims1'] = df['RegionGo'].apply(lambda x: [sim_consist(x,r) for r in focals])
df.head(2)

<ipython-input-7-41f946bc4508>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sims1'] = df['RegionGo'].apply(lambda x: [sim_consist(x,r) for r in focals])


,Absent1,Is_there,Dyad,Player,Round,Score,Region,RegionGo,Overlap,Sims1
5040,Unicorn_Absent,Unicorn_Absent,140-615,PL1,1,23,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.671875, 0.0, 0.6666666666666666, 0.20967741..."
5041,Unicorn_Absent,Unicorn_Absent,140-615,PL2,1,23,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.171875, 0.0, 0.16216216216216217, 0.1315789..."


In [8]:
df.to_csv('../Data/humans_absent.csv', index=False)

In [ ]:
pl = 0
modelParameters = [0.01, 0.01, 0.01, 0.01, 
                   1, 100, 31.5,
                   1, 100, 0.9,
                   1, 100, 0.9
                  ]
Num_Loc = 8
df_aux = df.copy()
df_aux['attract'] = df_aux.apply(lambda x: attractiveness(
    x['Region'], x['Score'], x['Overlap'], 
    pl, modelParameters, Num_Loc, focals), axis = 1
                        )
df_aux.head()

In [ ]:
df_aux['Err_L2'] = (df_aux['Sims'] - df_aux['attract'])**2
df_aux['Err_L2'] = df_aux['Err_L2'].apply(lambda x: np.sum(x))
df_aux.head()

In [ ]:
def err_l2(df, modelParameters):
    pl = 0
    Num_Loc = 8
    df['attract'] = df.apply(lambda x: attractiveness(
                            x['Region'], x['Score'], x['Overlap'], 
                            pl, modelParameters, Num_Loc, focals), 
                            axis = 1)
    df['Err_L2'] = (df['Sims'] - df['attract'])**2
    df['Err_L2'] = df['Err_L2'].apply(lambda x: np.sum(x))
    return df['Err_L2'].sum()

In [ ]:
err_l2(df, modelParameters)